# Deploy to Cloud AI Platform Prediction

In [1]:
import json
import os
import pandas as pd
import time

In [2]:
PREDICT_APP_V1_FOLDER = '../predict_app_v1'
PREDICT_APP_V2_FOLDER = '../predict_app_v2'
TESTING_DATA_PATH = 'gs://jk-demo-datasets/gasdata/testing.csv'
REGION = "us-central1"
ARTIFACT_BUCKET = "gs://jk-demo-artifacts"

## Create a model resource

In [3]:
MODEL_NAME = "octane_regression"
LABELS = "task=regression,domain=chemometrics"

In [ ]:
!gcloud ai-platform models create  $MODEL_NAME \
--regions=$REGION \
--labels=$LABELS

## Create a model version from a trained model

In [4]:
TRAINED_MODEL_PATH = 'gs://jk-demo-artifacts/models/octane_regressor_r14/'
MODEL_VERSION = 'v01'

In [ ]:
!gcloud ai-platform versions create $MODEL_VERSION \
--model=$MODEL_NAME \
--origin=$TRAINED_MODEL_PATH \
--runtime-version=1.14 \
--framework=scikit-learn \
--python-version=3.5

### Test the model version
#### Prepare a file with test instances

In [5]:
INPUT_FILE = '../datasets/testing_instances.json'

df_test = pd.read_csv(TESTING_DATA_PATH).drop('octane', axis=1)
with open(INPUT_FILE, "w") as f:
  for index, row in df_test.iterrows():
    f.write(json.dumps(list(row.values)))
    f.write("\n")

#### Run predictions

In [6]:
!gcloud ai-platform predict \
--model $MODEL_NAME \
--version $MODEL_VERSION \
--json-instances $INPUT_FILE

[89.24978589686916, 88.7120066196715, 84.58220954195754, 86.57325964405814, 88.29143761591263, 87.32235361510662]


## Create a model version from a custom prediction routine
### Create a predictor class

In [ ]:
os.makedirs(PREDICT_APP_V1_FOLDER, exist_ok=True)

In [ ]:
%%writefile $PREDICT_APP_V1_FOLDER/predict.py

import os
import numpy as np

from sklearn.externals import joblib

class OctaneRegressor(object):
    """A custom prediction routine for Octane regressor"""
    
    def __init__(self, model):
        """Stores the model loaded in from_path"""
        self._model = model
        
    def predict(self, instances, **kwargs):
        """Runs inference"""
    
        outputs = self._model.predict(np.asarray(instances))
        
        return outputs.tolist()

        
    @classmethod
    def from_path(cls, model_dir):
        """Loads the model from the joblib file"""
        model_path = os.path.join(model_dir, 'model.joblib')
        model = joblib.load(model_path)
        
        
        return cls(model)
    

### Create a source distribution package

In [ ]:
%%writefile $PREDICT_APP_V1_FOLDER/setup.py

from setuptools import setup

setup(
    name='custom-predictor-1',
    description='Custom prediction routine.',
    version='0.1',
    scripts=['predict.py']
)

In [ ]:
%cd $PREDICT_APP_V1_FOLDER
%run $PREDICT_APP_V1_FOLDER/setup.py sdist --formats=gztar
%cd -

### Copy the source distribution package to GCS

In [ ]:
CUSTOM_ROUTINE_NAME = "custom-predictor-1"
TARBALL_NAME = "custom-predictor-1-0.1.tar.gz"
LOCAL_PATH = "{}/dist/{}".format(PREDICT_APP_V1_FOLDER, TARBALL_NAME)
GCS_PATH = "{}/prediction_routines/{}/{}".format(ARTIFACT_BUCKET, CUSTOM_ROUTINE_NAME, TARBALL_NAME)

In [ ]:
!gsutil cp $LOCAL_PATH $GCS_PATH

### Create a model version

In [7]:
TRAINED_MODEL_PATH = 'gs://jk-demo-artifacts/models/octane_regressor_r14/'
MODEL_VERSION = 'v02'

In [ ]:
!gcloud beta ai-platform versions create $MODEL_VERSION \
--model=$MODEL_NAME \
--origin=$TRAINED_MODEL_PATH \
--runtime-version=1.14 \
--python-version=3.5 \
--package-uris $GCS_PATH \
--prediction-class predict.OctaneRegressor

### Test the model

In [8]:
!gcloud ai-platform predict \
--model $MODEL_NAME \
--version $MODEL_VERSION \
--json-instances $INPUT_FILE

[89.24978589686916, 88.7120066196715, 84.58220954195754, 86.57325964405814, 88.29143761591263, 87.32235361510662]


## Create a model version from a custom prediction routine with external dependencies
### Create a predictor class

In [12]:
os.makedirs(PREDICT_APP_V2_FOLDER, exist_ok=True)

In [13]:
%%writefile $PREDICT_APP_V2_FOLDER/predict.py

import os
import numpy as np

import joblib

class OctaneRegressor(object):
    """A custom prediction routine for Octane regressor"""
    
    def __init__(self, model):
        """Stores the model loaded in from_path"""
        self._model = model
        
    def predict(self, instances, **kwargs):
        """Runs inference"""
    
        outputs = self._model.predict(np.asarray(instances))
        
        return outputs.tolist()

        
    @classmethod
    def from_path(cls, model_dir):
        """Loads the model from the joblib file"""
        model_path = os.path.join(model_dir, 'model.joblib')
        model = joblib.load(model_path)
        
        
        return cls(model)
    

Overwriting ../predict_app_v2/predict.py


### Create a source distribution package

In [14]:
%%writefile $PREDICT_APP_V2_FOLDER/setup.py

from setuptools import setup

setup(
    name='custom-predictor-2',
    description='Custom prediction routine.',
    version='0.1',
    install_requires=[
      'scikit-learn>=0.21.2',
      'numpy>=1.11.0',
      'scipy>=0.17.0',
      'joblib>=0.11'
    ],
    scripts=['predict.py']
)


Overwriting ../predict_app_v2/setup.py


In [15]:
%cd $PREDICT_APP_V2_FOLDER
%run $PREDICT_APP_V2_FOLDER/setup.py sdist --formats=gztar
%cd -

/home/jupyter/caip-demo/predict_app_v2
running sdist
running egg_info
creating custom_predictor_2.egg-info
writing custom_predictor_2.egg-info/PKG-INFO
writing top-level names to custom_predictor_2.egg-info/top_level.txt
writing dependency_links to custom_predictor_2.egg-info/dependency_links.txt
writing requirements to custom_predictor_2.egg-info/requires.txt
writing manifest file 'custom_predictor_2.egg-info/SOURCES.txt'
reading manifest file 'custom_predictor_2.egg-info/SOURCES.txt'
writing manifest file 'custom_predictor_2.egg-info/SOURCES.txt'


running check


creating custom-predictor-2-0.1
creating custom-predictor-2-0.1/custom_predictor_2.egg-info
copying files to custom-predictor-2-0.1...
copying predict.py -> custom-predictor-2-0.1
copying setup.py -> custom-predictor-2-0.1
copying custom_predictor_2.egg-info/PKG-INFO -> custom-predictor-2-0.1/custom_predictor_2.egg-info
copying custom_predictor_2.egg-info/SOURCES.txt -> custom-predictor-2-0.1/custom_predictor_2.egg-info
copying custom_predictor_2.egg-info/dependency_links.txt -> custom-predictor-2-0.1/custom_predictor_2.egg-info
copying custom_predictor_2.egg-info/requires.txt -> custom-predictor-2-0.1/custom_predictor_2.egg-info
copying custom_predictor_2.egg-info/top_level.txt -> custom-predictor-2-0.1/custom_predictor_2.egg-info
Writing custom-predictor-2-0.1/setup.cfg
creating dist
Creating tar archive
removing 'custom-predictor-2-0.1' (and everything under it)
/home/jupyter/caip-demo/notebooks


### Copy the source distribution package to GCS

In [16]:
CUSTOM_ROUTINE_NAME = "custom-predictor-2"
TARBALL_NAME = "custom-predictor-2-0.1.tar.gz"
LOCAL_PATH = "{}/dist/{}".format(PREDICT_APP_V2_FOLDER, TARBALL_NAME)
GCS_PATH = "{}/prediction_routines/{}/{}".format(ARTIFACT_BUCKET, CUSTOM_ROUTINE_NAME, TARBALL_NAME)

In [17]:
!gsutil cp $LOCAL_PATH $GCS_PATH

Copying file://../predict_app_v2/dist/custom-predictor-2-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.1 KiB/  1.1 KiB]                                                
Operation completed over 1 objects/1.1 KiB.                                      


### Create a model version

In [18]:
TRAINED_MODEL_PATH = 'gs://jk-demo-artifacts/models/octane_regressor_container/'
MODEL_VERSION = 'v03'

In [19]:
!gcloud beta ai-platform versions create $MODEL_VERSION \
--model=$MODEL_NAME \
--origin=$TRAINED_MODEL_PATH \
--runtime-version=1.14 \
--python-version=3.5 \
--package-uris $GCS_PATH \
--prediction-class predict.OctaneRegressor

Creating version (this might take a few minutes)......done.                    


### Test the model

In [20]:
!gcloud ai-platform predict \
--model $MODEL_NAME \
--version $MODEL_VERSION \
--json-instances $INPUT_FILE

[89.24978589686916, 88.7120066196715, 84.58220954195754, 86.57325964405814, 88.29143761591263, 87.32235361510662]
